In [2]:
import pandas as pd
import ssl
import numpy as np
from pathlib import Path
import sqlalchemy
import datetime

# from utils.helpers import alpaca_symbol_data
from utils.prep_data import prep_data
from utils.zero_one import zero_one
from utils.one_large import one_large
from utils.negative import negative
# For creating the DB
database_connection_string = 'sqlite:///SP500.db'
engine = sqlalchemy.create_engine(database_connection_string)

# prevent an ssl failure
ssl._create_default_https_context = ssl._create_unverified_context

payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

# There are 2 tables on the Wikipedia page
# we want the first table
first_table = payload[0]
second_table = payload[1]

df500table = first_table

sp500Tickers = df500table['Symbol'].values.tolist()
df500sectors = df500table.drop(columns={'Security','SEC filings','GICS Sub-Industry', 'Headquarters Location', 'Date first added', 'CIK','Founded'})
df500sectors.set_index('Symbol', inplace=True)
sectors = df500sectors['GICS Sector'].unique()
# print(sp500Tickers)

# 1Y Data

The SPY is 1 for positive and 0 for negative

In [2]:
# Normal

StocksWide = pd.read_sql_table('StocksWide', engine)
SectorDF = prep_data(StocksWide, sectors, df500sectors)
SectorDF = zero_one(SectorDF)
SectorDF

C:\Users\dlpst\OneDrive\Documents\UC_Berkeley_Blockchain_Bootcamp\Projects\machine_learning_index_prediction\utils\prep_data.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SectorDF[sector] = TempDF.mean(axis=1)


,SPY Open,SPY Close,Industrials,Health Care,Information Technology,Communication Services,Consumer Staples,Consumer Discretionary,Utilities,Financials,Materials,Real Estate,Energy,SPY
timestamp,,,,,,,,,,,,,,
2021-04-19 04:00:00,413.93,412.17,-0.004149,-0.000356,-0.009452,-0.002727,-0.002429,-0.007847,-0.005995,-0.003404,-0.005562,0.002253,-0.001944,0.0
2021-04-20 04:00:00,411.55,416.07,-0.008910,0.002328,-0.005653,-0.007387,0.009700,-0.015166,0.012631,-0.011951,-0.008709,0.011858,-0.025533,1.0
2021-04-21 04:00:00,415.91,412.27,0.014615,0.010818,0.015466,0.016725,0.004558,0.019629,-0.006180,0.018827,0.019342,0.006578,0.034755,0.0
2021-04-22 04:00:00,412.82,416.74,-0.006297,-0.001288,-0.006342,-0.004689,-0.006656,-0.003878,-0.007886,-0.008718,-0.015059,-0.005400,-0.012065,1.0
2021-04-23 04:00:00,417.42,417.61,0.009747,0.007723,0.011177,0.003202,-0.002163,0.006801,-0.003110,0.020577,0.012318,0.003051,0.010520,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-08 04:00:00,444.11,439.92,-0.005791,0.005426,-0.007141,0.001863,0.001890,0.004482,0.001318,0.002899,0.002020,0.001696,0.027976,0.0
2022-04-11 04:00:00,443.02,438.29,-0.001636,-0.016449,-0.008222,-0.001345,-0.002333,0.006947,-0.014041,-0.002284,-0.001462,-0.010802,-0.018648,0.0
2022-04-12 04:00:00,437.96,443.31,-0.005875,-0.009887,-0.015830,-0.014729,0.000453,-0.009666,0.006666,-0.009787,-0.003684,0.001091,-0.003254,1.0


In [3]:
SectorDF.to_sql('SectorDF', engine, if_exists='replace')

# 1Y Negative

SPY > 0 then 1

SPY < 0 then -1

In [4]:
StocksWide = pd.read_sql_table('StocksWide', engine)
SectorDFNegative = prep_data(StocksWide, sectors, df500sectors)
SectorDFNegative = one_large(SectorDFNegative)
SectorDFNegative

C:\Users\dlpst\OneDrive\Documents\UC_Berkeley_Blockchain_Bootcamp\Projects\machine_learning_index_prediction\utils\prep_data.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SectorDF[sector] = TempDF.mean(axis=1)


,SPY Open,SPY Close,Industrials,Health Care,Information Technology,Communication Services,Consumer Staples,Consumer Discretionary,Utilities,Financials,Materials,Real Estate,Energy,SPY
timestamp,,,,,,,,,,,,,,
2021-04-19 04:00:00,413.93,412.17,-0.004149,-0.000356,-0.009452,-0.002727,-0.002429,-0.007847,-0.005995,-0.003404,-0.005562,0.002253,-0.001944,0.0
2021-04-20 04:00:00,411.55,416.07,-0.008910,0.002328,-0.005653,-0.007387,0.009700,-0.015166,0.012631,-0.011951,-0.008709,0.011858,-0.025533,1.0
2021-04-21 04:00:00,415.91,412.27,0.014615,0.010818,0.015466,0.016725,0.004558,0.019629,-0.006180,0.018827,0.019342,0.006578,0.034755,-1.0
2021-04-22 04:00:00,412.82,416.74,-0.006297,-0.001288,-0.006342,-0.004689,-0.006656,-0.003878,-0.007886,-0.008718,-0.015059,-0.005400,-0.012065,1.0
2021-04-23 04:00:00,417.42,417.61,0.009747,0.007723,0.011177,0.003202,-0.002163,0.006801,-0.003110,0.020577,0.012318,0.003051,0.010520,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-08 04:00:00,444.11,439.92,-0.005791,0.005426,-0.007141,0.001863,0.001890,0.004482,0.001318,0.002899,0.002020,0.001696,0.027976,-1.0
2022-04-11 04:00:00,443.02,438.29,-0.001636,-0.016449,-0.008222,-0.001345,-0.002333,0.006947,-0.014041,-0.002284,-0.001462,-0.010802,-0.018648,-1.0
2022-04-12 04:00:00,437.96,443.31,-0.005875,-0.009887,-0.015830,-0.014729,0.000453,-0.009666,0.006666,-0.009787,-0.003684,0.001091,-0.003254,1.0


In [5]:
SectorDFNegative.to_sql('SectorDFNegative', engine, if_exists='replace')

# 1Y Larger Seperation


SPY > .005 then 1

SPY < -.005 then -1

in between 0

In [6]:
# Large
StocksWide = pd.read_sql_table('StocksWide', engine)
SectorDFLarge = prep_data(StocksWide, sectors, df500sectors)
SectorDFLarge = one_large(SectorDFLarge)
SectorDFLarge

C:\Users\dlpst\OneDrive\Documents\UC_Berkeley_Blockchain_Bootcamp\Projects\machine_learning_index_prediction\utils\prep_data.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SectorDF[sector] = TempDF.mean(axis=1)


,SPY Open,SPY Close,Industrials,Health Care,Information Technology,Communication Services,Consumer Staples,Consumer Discretionary,Utilities,Financials,Materials,Real Estate,Energy,SPY
timestamp,,,,,,,,,,,,,,
2021-04-19 04:00:00,413.93,412.17,-0.004149,-0.000356,-0.009452,-0.002727,-0.002429,-0.007847,-0.005995,-0.003404,-0.005562,0.002253,-0.001944,0.0
2021-04-20 04:00:00,411.55,416.07,-0.008910,0.002328,-0.005653,-0.007387,0.009700,-0.015166,0.012631,-0.011951,-0.008709,0.011858,-0.025533,1.0
2021-04-21 04:00:00,415.91,412.27,0.014615,0.010818,0.015466,0.016725,0.004558,0.019629,-0.006180,0.018827,0.019342,0.006578,0.034755,-1.0
2021-04-22 04:00:00,412.82,416.74,-0.006297,-0.001288,-0.006342,-0.004689,-0.006656,-0.003878,-0.007886,-0.008718,-0.015059,-0.005400,-0.012065,1.0
2021-04-23 04:00:00,417.42,417.61,0.009747,0.007723,0.011177,0.003202,-0.002163,0.006801,-0.003110,0.020577,0.012318,0.003051,0.010520,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-08 04:00:00,444.11,439.92,-0.005791,0.005426,-0.007141,0.001863,0.001890,0.004482,0.001318,0.002899,0.002020,0.001696,0.027976,-1.0
2022-04-11 04:00:00,443.02,438.29,-0.001636,-0.016449,-0.008222,-0.001345,-0.002333,0.006947,-0.014041,-0.002284,-0.001462,-0.010802,-0.018648,-1.0
2022-04-12 04:00:00,437.96,443.31,-0.005875,-0.009887,-0.015830,-0.014729,0.000453,-0.009666,0.006666,-0.009787,-0.003684,0.001091,-0.003254,1.0


In [7]:

SectorDFLarge.to_sql('SectorDFLarge', engine, if_exists='replace')

# 3Y Data

In [6]:
StocksWide3Y = pd.read_sql_table('StocksWide3Y', engine)
StocksWide3Y

,index,timestamp,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,...,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS,SPY Open,SPY Close
0,0,2019-04-15 04:00:00,-0.007407,-0.025974,0.004930,0.003526,0.001607,0.013989,0.003131,-0.007174,...,-0.001980,-0.004469,-0.003893,0.000394,-0.004127,-0.008318,-0.015292,-0.005890,290.27,289.97
1,1,2019-04-16 04:00:00,-0.040460,0.002061,-0.014297,-0.000351,-0.011705,-0.012321,-0.029202,-0.023523,...,0.004702,-0.018890,0.003408,-0.003627,-0.026297,0.007138,0.009623,-0.028602,290.96,290.16
2,2,2019-04-17 04:00:00,-0.034805,-0.002032,-0.017449,0.019320,-0.026102,-0.043524,-0.060132,-0.041053,...,0.000123,-0.035792,-0.000966,-0.003730,-0.048872,-0.013509,-0.003092,-0.014385,291.41,289.45
3,3,2019-04-18 04:00:00,0.005810,-0.001743,-0.006570,0.002015,-0.005640,0.002075,0.016079,0.005714,...,-0.005516,0.014691,0.000000,0.015929,0.002841,0.003796,-0.021109,0.007631,290.12,290.02
4,4,2019-04-22 04:00:00,-0.004741,-0.018182,-0.025050,0.009227,0.007217,-0.018915,0.020454,0.019010,...,0.012581,-0.002428,0.007968,0.001748,0.015841,-0.004319,-0.008070,0.014182,289.17,290.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,753,2022-04-08 04:00:00,-0.006190,-0.016598,0.037742,-0.009665,0.011329,0.011742,-0.002632,-0.001377,...,0.016624,0.000625,-0.003106,0.003790,-0.002386,-0.001513,0.003298,0.011628,448.04,447.57
754,754,2022-04-11 04:00:00,-0.014713,0.028485,-0.021131,-0.017894,-0.029543,-0.018407,-0.042728,-0.027859,...,-0.023524,0.026032,-0.009245,-0.005213,-0.017041,-0.005064,0.004221,-0.032517,444.11,439.92
755,755,2022-04-12 04:00:00,-0.047840,0.005282,-0.001260,-0.001905,-0.011813,0.008872,-0.011187,-0.011687,...,-0.004651,-0.000413,-0.014289,0.007479,-0.003776,-0.021101,-0.016490,-0.021862,443.02,438.29
756,756,2022-04-13 04:00:00,0.024190,0.060436,-0.005142,0.018225,-0.036667,0.002029,0.013956,0.016332,...,0.000692,0.007440,0.023294,0.015922,0.015073,0.009472,0.023338,0.008139,437.96,443.31


In [7]:
StocksWide3Y = pd.read_sql_table('StocksWide3Y', engine)
SectorDF3Y = prep_data(StocksWide3Y, sectors, df500sectors)
SectorDF3Y = zero_one(SectorDF3Y)
SectorDF3Y.to_sql('SectorDF3Y', engine, if_exists='replace')
SectorDF3Y

c:\Users\canav\Documents\Fintech\machine_learning_index_prediction\utils\prep_data.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SectorDF[sector] = TempDF.mean(axis=1)


,SPY Open,SPY Close,Industrials,Health Care,Information Technology,Communication Services,Consumer Staples,Consumer Discretionary,Utilities,Financials,Materials,Real Estate,Energy,SPY
timestamp,,,,,,,,,,,,,,
2019-04-15 04:00:00,290.96,290.16,-0.004377,0.001836,-0.002121,0.000174,0.004429,-0.000375,-0.002463,-0.005410,-0.004234,-0.006263,-0.007864,0.0
2019-04-16 04:00:00,291.41,289.45,0.002411,-0.033770,0.003420,-0.004447,-0.003661,0.001813,-0.012484,0.012432,0.006651,-0.023859,0.003874,0.0
2019-04-17 04:00:00,290.12,290.02,-0.005798,-0.037141,-0.008803,0.000684,-0.001941,-0.003729,-0.003592,-0.003355,-0.012572,-0.014445,-0.006295,0.0
2019-04-18 04:00:00,289.17,290.27,0.003305,0.003877,-0.001119,-0.000434,0.000125,-0.000186,-0.001080,-0.001865,0.003768,0.008753,-0.007930,1.0
2019-04-22 04:00:00,290.68,292.88,-0.000389,0.003974,0.004205,0.002713,0.000675,-0.004686,-0.000330,0.000803,-0.002956,-0.008885,0.013080,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-07 04:00:00,448.04,447.57,0.005856,0.021049,0.005208,-0.009337,0.008555,0.004695,-0.003217,-0.001480,0.004416,-0.004456,0.002626,0.0
2022-04-08 04:00:00,444.11,439.92,-0.005791,0.005426,-0.007141,0.001863,0.001890,0.004482,0.001318,0.002899,0.002020,0.001696,0.027976,0.0
2022-04-11 04:00:00,443.02,438.29,-0.001636,-0.016449,-0.008222,-0.001345,-0.002333,0.006947,-0.014041,-0.002284,-0.001462,-0.010802,-0.018648,0.0


In [8]:
StocksWide3Y = pd.read_sql_table('StocksWide3Y', engine)
SectorDF3Y = prep_data(StocksWide3Y, sectors, df500sectors)
SectorDF3YLarge = one_large(SectorDF3Y)
SectorDF3YLarge['SPY'].value_counts()

c:\Users\canav\Documents\Fintech\machine_learning_index_prediction\utils\prep_data.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SectorDF[sector] = TempDF.mean(axis=1)


 0.0    404
 1.0    192
-1.0    161
Name: SPY, dtype: int64

In [9]:
SectorDF3YLarge.to_sql('SectorDF3YLarge', engine, if_exists='replace')

In [10]:
StocksWide3Y = pd.read_sql_table('StocksWide3Y', engine)
SectorDF3Y = prep_data(StocksWide3Y, sectors, df500sectors)
SectorDF3YNegative = negative(SectorDF3Y)
SectorDF3YNegative['SPY'].value_counts()

c:\Users\canav\Documents\Fintech\machine_learning_index_prediction\utils\prep_data.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SectorDF[sector] = TempDF.mean(axis=1)


 1.0    410
-1.0    341
 0.0      6
Name: SPY, dtype: int64

In [11]:
SectorDF3YNegative.to_sql('SectorDF3YNegative', engine, if_exists='replace')

# 5 Day Moving Average

Work in process, do not reference past this

In [13]:
StocksWide = pd.read_sql_table('StocksWide', engine)
StocksWide.head()

,index,timestamp,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,...,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS,SPY Open,SPY Close
0,0,2021-04-19 04:00:00,-0.006549,-0.019563,-0.005424,0.010110,0.005928,-0.004167,0.003415,0.007524,...,-0.005634,0.000611,-0.006974,-0.005660,0.003548,-0.007537,-0.004802,0.003582,416.26,415.21
1,1,2021-04-20 04:00:00,-0.004060,-0.030938,-0.020657,-0.013926,0.008417,0.010059,-0.002388,-0.012020,...,-0.016717,0.001989,-0.005638,0.007258,0.007419,-0.011536,-0.050245,0.004704,413.93,412.17
2,2,2021-04-21 04:00:00,0.007149,0.050500,0.027873,0.008614,0.009199,0.001322,0.032889,0.018019,...,0.027523,0.011623,0.017189,-0.002632,0.014997,0.009901,0.030146,0.003361,411.55,416.07
3,3,2021-04-22 04:00:00,0.003731,-0.069541,0.006533,-0.006850,-0.005423,-0.004857,0.007363,0.005618,...,-0.011270,-0.016418,-0.008364,-0.003046,0.004531,-0.017499,-0.021140,-0.003771,415.91,412.27
4,4,2021-04-23 04:00:00,0.015680,0.041441,-0.006120,0.015652,0.015222,0.006536,0.015741,-0.002508,...,0.005246,0.011305,0.007570,-0.000085,0.003389,0.011777,0.035489,0.009112,412.82,416.74


In [14]:
WideDF.drop(columns='index', inplace=True)
WideDF = WideDF.set_index('timestamp')
SectorDF = WideDF[['SPY', 'SPY Open', 'SPY Close']]

stockstodrop = []
StocksWideNoSPY = WideDF.drop(columns={'SPY','SPY Open','SPY Close'})
for sector in sectors:
    for (columnName, columnValues) in StocksWideNoSPY.iteritems():
        if sector != df500sectors.loc[columnName]['GICS Sector']:
            stockstodrop.append(columnName)
    TempDF = StocksWideNoSPY.drop(columns=stockstodrop)
    stockstodrop = []
    SectorDF[sector] = TempDF.mean(axis=1)
SectorDF

NameError: name 'WideDF' is not defined

In [ ]:
# SP500_Constituents_details = pd.read_csv(Path("resources/constituents_financials.csv"), index_col='Symbol')
# SP500_Constituents_details = SP500_Constituents_details.sort_values(by=['Market Cap'], ascending=False)
# SP500_Constituents_details.head()

In [ ]:
# StockCount = 0
# Sleeve = 1
# StockMarketCap = [[],[],[],[],[],[]]

# for index, row in SP500_Constituents_details.iterrows():
#     if StockCount == Sleeve * 100:
#         Sleeve += 1
#     StockMarketCap[Sleeve-1].append(index)
#     StockCount += 1